Идея данного базового решения - повторение списка продуктов из последнего известного заказа

In [1]:
import pandas as pd

In [2]:
path_to_data = '..\\data\\'

In [3]:
orders_df = pd.read_csv(path_to_data + 'orders.csv')

In [6]:
orders_df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
test_orders_df = orders_df[orders_df['eval_set'] == 'test']

In [5]:
test_orders_df

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0
...,...,...,...,...,...,...,...
3420918,2728930,206202,test,23,2,17,6.0
3420929,350108,206204,test,5,4,14,14.0
3421001,1043943,206206,test,68,0,20,0.0
3421018,2821651,206207,test,17,2,13,14.0


In [20]:
prev_order = (test_orders_df.groupby(['order_id', 'user_id'])['order_number'].max() - 1).reset_index()

In [21]:
prev_order

,order_id,user_id,order_number
0,17,36855,4
1,34,35220,19
2,137,187107,8
3,182,115892,27
4,257,35581,8
...,...,...,...
74995,3420740,195822,72
74996,3420877,15955,19
74997,3420888,162374,7
74998,3420989,109358,6


In [22]:
prev_order = orders_df.merge(prev_order, how='inner', left_on=['user_id', 'order_number'], right_on=['user_id', 'order_number'], suffixes=('_train', '_test'))

In [23]:
prev_order

,order_id_train,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_id_test
0,1402502,3,prior,12,1,15,15.0,2774568
1,2557754,4,prior,5,5,13,0.0,329954
2,998866,6,prior,3,2,18,12.0,1528013
3,1468214,11,prior,7,5,9,30.0,1376945
4,221248,12,prior,5,1,9,30.0,1356845
...,...,...,...,...,...,...,...,...
74995,1112560,206202,prior,22,3,14,6.0,2728930
74996,1380497,206204,prior,4,4,15,25.0,350108
74997,1904200,206206,prior,67,0,13,11.0,1043943
74998,1005822,206207,prior,16,2,7,18.0,2821651


In [26]:
target_orders = prev_order['order_id_train'].unique()

In [24]:
prior_df = pd.read_csv(path_to_data + 'order_products__prior.csv')

In [25]:
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [27]:
target_prior_df = prior_df[prior_df['order_id'].isin(target_orders)]

In [29]:
target_prior_df.head()

,order_id,product_id,add_to_cart_order,reordered
404,51,30274,1,0
405,51,3594,2,1
406,51,14994,3,0
407,51,45433,4,0
408,51,44514,5,0


In [31]:
previous_products_df = target_prior_df.groupby(['order_id'])['product_id'].apply(list).reset_index(name='new')

In [32]:
previous_products_df

,order_id,new
0,51,"[30274, 3594, 14994, 45433, 44514, 32624, 604,..."
1,89,"[17794, 20600, 5876, 37011, 26648]"
2,94,"[35221, 16521, 11520, 17948]"
3,132,"[36640, 17980, 8239, 17830, 39871, 19677, 4329..."
4,222,"[24852, 47626, 35522, 21903, 25043, 8424, 2820..."
...,...,...
74995,3420885,"[37274, 21070, 47144, 33574, 35134, 36534, 412..."
74996,3420956,"[40001, 28204]"
74997,3420963,"[47209, 27392, 39475]"
74998,3421005,"[14419, 35269, 4938]"


In [46]:
target_df = prev_order.merge(previous_products_df, how='inner', left_on='order_id_train', right_on='order_id')[['order_id_test', 'new']]

In [47]:
target_df

,order_id_test,new
0,2774568,"[39190, 18599, 23650, 21903, 47766, 24810]"
1,329954,"[26576, 25623, 21573]"
2,1528013,"[49401, 25659, 8424]"
3,1376945,"[24799, 17706, 33572, 27959, 48697, 49374, 830..."
4,1356845,"[13176, 14992, 44422, 11520, 31506, 22959, 712..."
...,...,...
74995,2728930,"[26683, 5194, 38695, 1067, 35221, 21709, 41177..."
74996,350108,"[43961, 15649, 10199, 5646, 20995, 33754, 1852..."
74997,1043943,"[11520, 23029, 42623]"
74998,2821651,"[27845, 36011, 39180, 13176, 33787, 44632, 337..."


In [48]:
target_df['products'] = target_df['new'].apply(lambda x: [str(t) for t in x]).apply(lambda x: ' '.join(x))

In [51]:
target_df = target_df.rename(columns={'order_id_test': 'order_id'})

In [58]:
target_df[['order_id', 'products']].sort_values(['order_id'])

,order_id,products
13407,17,13107 26429 39275 21463
12775,34,44912 47766 16083 2596 37630 23028 39475 13176...
68056,137,13740 34126 44422 29594 25890 24852 41787 2379...
42239,182,37642 5621 5479 47672 9337 39275 32109 40377 1...
12915,257,49235 24852 30233 29837 37646 27104 47547 3947...
...,...,...
71157,3420740,2314 1158 48070 28476 19660 17872 35951 12597 ...
5753,3420877,3661 8136 32293 42450 13646 12238 5161 49111 2...
59190,3420888,43961 46906 10880 35951 44632 10111 20542
39824,3420989,46676 35004 5134 45866 42450 17948 39075 8555 ...


In [59]:
target_df[['order_id', 'products']].sort_values(['order_id']).to_csv(path_to_data + 'submissions\\baseline.csv', index=False)

Результат этого сабмита: 0.31180 Public; 0.31202 Private;